In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import pickle
import warnings

from scipy import stats

# Ignore all warnings
warnings.filterwarnings('ignore')

In [3]:
with open('results_regression_test.pkl', 'rb') as f:
    results = pickle.load(f)

In [4]:
setup_names = {}
base_names = ['LIME', 'SHAP', 'FCER', 'CCER', 'PFCER', 'PCCER']
norm_names = ['', ' Dist', ' Std', ' Abs', ' Var']
for i, base in enumerate(['lime', 'shap', 'ce', 'cce', 'pce', 'pcce']):
    for j, setup in enumerate(['', '_dist', '_std', '_abs', '_var']):
        setup_names[base + setup] = base_names[i] + norm_names[j]
setup_names['lime_base'] = 'LIME Base'
setup_names['shap_base'] = 'SHAP Base'
setup_names

{'lime': 'LIME',
 'lime_dist': 'LIME Dist',
 'lime_std': 'LIME Std',
 'lime_abs': 'LIME Abs',
 'lime_var': 'LIME Var',
 'shap': 'SHAP',
 'shap_dist': 'SHAP Dist',
 'shap_std': 'SHAP Std',
 'shap_abs': 'SHAP Abs',
 'shap_var': 'SHAP Var',
 'ce': 'FCER',
 'ce_dist': 'FCER Dist',
 'ce_std': 'FCER Std',
 'ce_abs': 'FCER Abs',
 'ce_var': 'FCER Var',
 'cce': 'CCER',
 'cce_dist': 'CCER Dist',
 'cce_std': 'CCER Std',
 'cce_abs': 'CCER Abs',
 'cce_var': 'CCER Var',
 'pce': 'PFCER',
 'pce_dist': 'PFCER Dist',
 'pce_std': 'PFCER Std',
 'pce_abs': 'PFCER Abs',
 'pce_var': 'PFCER Var',
 'pcce': 'PCCER',
 'pcce_dist': 'PCCER Dist',
 'pcce_std': 'PCCER Std',
 'pcce_abs': 'PCCER Abs',
 'pcce_var': 'PCCER Var',
 'lime_base': 'LIME Base',
 'shap_base': 'SHAP Base'}

In [5]:
stab_timer = results['housing']['RF']['stab_timer']
rob_timer = results['housing']['RF']['rob_timer']

stab = 'Stability'
rob = 'Robustness'
timer = {}
timer[stab] = {}
timer[rob] = {}
for setup in stab_timer.keys():
    timer[stab][setup_names[setup]] = np.mean(stab_timer[setup]) / 10 # number of instances 
    timer[rob][setup_names[setup]] = np.mean(rob_timer[setup]) / 10 # number of instances 

timer = pd.DataFrame(timer)
pd.options.display.float_format = "{:.3f}".format
timer

,Stability,Robustness
LIME Base,0.137,0.162
SHAP Base,0.383,0.391
LIME,0.150,0.162
LIME Dist,0.223,0.241
LIME Std,0.305,0.331
LIME Abs,0.230,0.247
LIME Var,0.258,0.283
SHAP,0.517,0.547
SHAP Dist,0.967,0.944
SHAP Std,1.716,1.738


In [6]:
stability = results['housing']['RF']['stability']
robustness = results['housing']['RF']['robustness']

stab = 'Stability'
rob = 'Robustness'
performance = {}
performance[stab] = {}
performance[rob] = {}
performance['Prediction Variance'] = np.mean([np.var([robustness['predict'][i][j] for i in range(len(robustness['predict']))]) for j in range(len(robustness['predict'][0]))])
try:
    for setup in stability.keys():
        # Get the most important feature for each instance using the absolute value of the feature weights
        stab_feature = []
        rob_feature = []
        for j in range(len(stability[setup][0])): # number of instances        
            stab_feature.append([np.argmax(np.abs(stability[setup][i][j]['predict'])) for i in range(len(stability[setup]))])
            rob_feature.append([np.argmax(np.abs(robustness[setup][i][j]['predict'])) for i in range(len(robustness[setup]))])
        stab_feature = stats.mode(stab_feature, axis=1)
        rob_feature = stats.mode(rob_feature, axis=1)
        
        
        performance[stab][setup_names[setup]] = np.mean([np.var([stability[setup][i][j]['predict'][stab_feature.mode[j]] for i in range(len(stability[setup]))]) for j in range(len(stability[setup][0]))])
        performance[rob][setup_names[setup]] = np.mean([np.var([robustness[setup][i][j]['predict'][rob_feature.mode[j]] for i in range(len(robustness[setup]))]) for j in range(len(robustness[setup][0]))])
except:
    print(setup, robustness[setup])

performance = pd.DataFrame(performance)
pd.options.display.float_format = "{:.3e}".format
performance

,Stability,Robustness,Prediction Variance
LIME Base,2.957e-05,8.813e-04,9.069e-05
SHAP Base,1.719e-33,1.425e-04,9.069e-05
LIME,2.919e-05,8.711e-04,9.069e-05
LIME Dist,2.924e-05,8.533e-04,9.069e-05
LIME Std,3.092e-05,8.539e-04,9.069e-05
LIME Abs,2.784e-05,8.659e-04,9.069e-05
LIME Var,2.411e-05,8.817e-04,9.069e-05
SHAP,6.211e-34,1.425e-04,9.069e-05
SHAP Dist,7.704e-34,1.412e-04,9.069e-05
SHAP Std,1.103e-33,1.409e-04,9.069e-05
